In [26]:
import pymunk
import random
import pygame
import cv2
import numpy as np
import os
import pymunk.pygame_util 
import time
import json
import csv

In [27]:
img_dir     = "/home/turgay/falling_objects_dataset/img_files/"  
csv_dir     = "/home/turgay/falling_objects_dataset/csv_files/"

In [28]:
world_width  = 512
world_height = 512
xCenter      = world_width / 2

In [29]:
def dim():
    return random.randint(50, 100)

def xpos(prev_block):
    prev_w = prev_block[2]  # width of the previous block
    prev_x = prev_block[0]  # X position of the previous block
    return random.randint(prev_x - prev_w, prev_x + prev_w)

def ypos(prev_block, h):
    prev_y = prev_block[1]  # Y position of the previous block
    prev_h = prev_block[3]  # height of the previous block
    return prev_y - (prev_h)


def add_block(prev_block, is_first):
    w = dim()
    h = dim()
    x = xCenter if is_first else xpos(prev_block)
    y = ypos(prev_block, h)

    return (x, y, w, h)

def make_tower_world():
    
    ground_block = (xCenter, world_height - 10, world_width, 10)
    
    block1 = add_block(ground_block, True)
    block2 = add_block(block1, False)
    block3 = add_block(block2, False)
    block4 = add_block(block3, False)
    block5 = add_block(block4, False)
    return [ground_block, block1, block2, block3, block4, block5]

In [30]:
ground_block = (xCenter, world_height - 10, world_width, 10)

In [31]:
def save_img(batch, iteration, screen):
    
#---------------------------------------------------------------------------------------------
    pygame.image.save(screen, f"{img_dir}/{np.int64(iteration / 50) + (batch * 20) }.png")
#---------------------------------------------------------------------------------------------            

In [32]:
def save_csv(block_data, batch, iteration):
    
#---------------------------------------------------------------------------------------------    
    block_data2 = []
    
    blocks_for_csv        =   [data["block"] for data in block_data[:5]]
    masses_for_csv        =   [data["mass"] for data in block_data[:5]]
    color_r_for_csv       =   [data["color_r"] for data in block_data[:5]]   
    color_g_for_csv       =   [data["color_g"] for data in block_data[:5]]
    color_b_for_csv       =   [data["color_b"] for data in block_data[:5]]
        
    for block, mass, color_r, color_g, color_b in zip(blocks_for_csv, masses_for_csv, color_r_for_csv, color_g_for_csv, color_b_for_csv):
        data = {
            "position_x": block[0],
            "position_y": block[1],
            "width": block[2],
            "height": block[3],
            "mass": mass, 
            "color_r": color_r,
            "color_g": color_g,
            "color_b": color_b,
            "time"      : iteration / 50
        }
        block_data2.append(data)

#---------------------------------------------------------------------------------------------    
    
    csv_filename = f"{csv_dir}/{np.int64(iteration / 50) + (batch * 20)}.csv"
    
    # Check if CSV file exists to append or write headers
    file_exists = os.path.exists(csv_filename)
    with open(csv_filename, 'a', newline='') as csvfile:
        fieldnames = ["position_x", "position_y", "width", "height", "mass", "color_r", "color_g", "color_b", "time"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write headers only if file doesn't exist
        if not file_exists:
            writer.writeheader()
        
        # Write the data of the blocks to the CSV file
        writer.writerows(block_data2)


In [33]:
def start_video(batch, fps=60):

    video_path   =   f"{video_dir}/{np.int64(batch) }.avi"
    
    video_writer = cv2.VideoWriter(
                                    video_path,
                                    cv2.VideoWriter_fourcc(*'XVID'),
                                    fps,
                                    (world_width, world_height)
                                    )
    return video_writer

In [34]:
def record_frames(video_writer, screen):

    frame = pygame.surfarray.array3d(screen)  # Get frame data from Pygame
    frame = np.transpose(frame, (1, 0, 2))    # Transpose to match OpenCV format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert to OpenCV color format
    video_writer.write(frame)  # Write frame to video file

In [35]:
def stop_video(video_writer):
    video_writer.release()

In [36]:
def animate_tower(batch):
    
    pygame.init()
    
    space         = pymunk.Space()
    space.gravity = (0, 9.8)

    screen        = pygame.display.set_mode((world_width, world_height))
    draw_options  = pymunk.pygame_util.DrawOptions(screen)

    # Add ground
    ground = pymunk.Segment(space.static_body, (0, world_height - 10), (world_width, world_height - 10), 1)
    space.add(ground)

    # Create blocks for the tower
    blocks     = make_tower_world()
    block_data = []

    for block in blocks:
        
        mass          = random.randint(1, 10)
        moment        = pymunk.moment_for_box(mass, (block[2], block[3]))   # mass, size(width, height)   obj= (x, y, w, h, c) 
        body          = pymunk.Body(mass, moment)
        body.position = (block[0], block[1]) 

        shape   = pymunk.Poly.create_box(body, (block[2], block[3]))
        
        r, g, b = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        shape.color = (r , g, b, 1.0)

        space.add(body, shape)
        block_data.append({"mass": mass, "block": block, "color_r": r, "color_g": g, "color_b": b })

    
    iteration = 1
    while iteration <= 1000:  
                   
        screen.fill((255, 255, 255))  
        space.debug_draw(draw_options)
        pygame.display.flip()

        if iteration % 50  == 0:
                
            save_img(batch, iteration, screen)
            save_csv(block_data, batch, iteration)

        space.step(1 / 60.0)  # Step the physics simulation
        iteration += 1
        

    pygame.quit()

In [37]:
for i in range():
    
    animate_tower(i) 

